In [1]:
impor tglob
from sort import *
import numpy as np

/home/jvwoerden/.local/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [2]:
video_paths = glob.glob('../data/Fudan/*/*')

In [23]:
import json
# Get all the frames of a single video (all frames in given directory)
def get_all_frames(path):
    files = glob.glob('{}/*.json'.format(path))
    files.sort()
    ret = []
    for file in files:
        with open(file) as json_file:
            data = json.load(json_file)
            
        # Fix weird json syntax in file
        data = data[list(data.keys())[0]]
        t = {}
        t['filename'] = '{}/{}'.format(path, data['filename'])
        t['centers'] = []
        t['bboxes'] = []
        
        for region in data['regions']:
            
            t['bboxes'].append([
                region['shape_attributes']['x'],
                region['shape_attributes']['y'],
                region['shape_attributes']['x'] + region['shape_attributes']['width'],
                region['shape_attributes']['y'] + region['shape_attributes']['height'],
            ])
            
            t['centers'].append({
                'x': region['shape_attributes']['x'] + region['shape_attributes']['width']/2,
                'y': region['shape_attributes']['y'] + region['shape_attributes']['height']/2,
            })
            
        ret.append(t)
    return ret

In [24]:
from PIL import ImageDraw, Image, ImageFont

def get_img_with_centers(frame, track_id = False):
    img = Image.open(frame['filename'])
    draw = ImageDraw.Draw(img)

    for i, region in enumerate(frame['centers']):
        r = 4
        leftUpPoint = (region['x']-r, region['y']-r)
        rightDownPoint = (region['x']+r, region['y']+r)
        twoPointList = [leftUpPoint, rightDownPoint]
        draw.ellipse(twoPointList, fill=(255,0,0,255))
        if track_id:
            leftUpPoint = (region['x']-r+8, region['y']-r+8)
            draw.text(leftUpPoint, str(int(frame['tracked_id'][i])))
        
    return img

In [5]:
#get_img_with_centers(get_all_frames(video_paths[0])[0])

In [40]:
import copy

def get_track_frames(frames):
    frames = copy.deepcopy(frames)
    mot_tracker = Sort()
    for i, frame_info in enumerate(frames):
        original_bboxes = np.array(frame_info['bboxes'])
        
        track_bbs_ids = mot_tracker.update(original_bboxes)
        frames[i]['tracked_id'] = [int(i) for i in track_bbs_ids[:,-1].tolist()]
        
        bboxes = []
        centers = []
        
        for item in track_bbs_ids:
            listed_item = item.tolist()
            bboxes.append(listed_item)
            centers.append({
                'x': (listed_item[0] + listed_item[2]) / 2,
                'y': (listed_item[1] + listed_item[3]) / 2
            })
            
        frames[i]['bboxes'] = bboxes
        frames[i]['centers'] = centers
    return frames

In [47]:
frames = get_all_frames(video_paths[2])
frames = get_track_frames(frames)

/home/jvwoerden/.local/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


In [48]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import matplotlib.animation as animation
from IPython.core.display import display, HTML
from tqdm.notebook import tqdm
from IPython.display import clear_output
def generate_video(frame_info):
    
    c_frames = len(frame_info)

    fig,ax = plt.subplots(1, figsize=(14,8))
    fig.tight_layout()
    plt.axis('off')

    pbar = tqdm(total=c_frames)
    pbar.set_description("Generating frames")
    imgs = []
    for frame in frame_info[0:c_frames]:
        img = get_img_with_centers(frame, track_id = True)

        imgs.append([ax.imshow(img)])
        pbar.update(1)
    pbar.close()

    pbar = tqdm(total=1)
    pbar.set_description("Stitching frames together")
    ani = animation.ArtistAnimation(fig, imgs, interval=40, blit=True)
    pbar.update(1)
    pbar.close()

    pbar = tqdm(total=1)
    pbar.set_description("Prepare for display")
    tag = ani.to_html5_video()
    pbar.update(1)
    pbar.close()
    plt.close()

    clear_output()
    display(HTML(tag))

In [49]:
generate_video(frames)